<b>1. Import libraries</b>

In [1]:
from pytrends.request import TrendReq
import pandas as pd
import plotly
import plotly.express as px
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline 

from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

<b>2. Initialize</b>

In [2]:
pytrends = TrendReq()

<b>3. General settings</b>

In [3]:
keywords = ['python']
cat = '0'
geo = 'ID'
gprop = 'youtube'

timeframes = [
    'today 5-y',
    'today 3-y',
    'today 12-m',
    'today 5-m',
    'today 3-m', 
    'today 1-m',
    'now 1-d',
    'now 7-d',
    'now 1-H',
    'now 4-H'
]

In [4]:
pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=timeframes[4])

<b>4. Timeseries trends data</b>

In [5]:
# pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=timeframes[4])
timeseries = pytrends.interest_over_time()
timeseries.reset_index(inplace=True)
timeseries.rename(columns={'date':'datetime', keywords[0]: 'value', 'isPartial':'partial'}, inplace=True)
timeseries.head()

,datetime,value,partial
0,2021-12-01,39,False
1,2021-12-02,76,False
2,2021-12-03,48,False
3,2021-12-04,32,False
4,2021-12-05,53,False


In [6]:
from bokeh.plotting import figure, show
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter
from bokeh.layouts import column
from bokeh.models import HoverTool

# prepare some data
x = timeseries.datetime
y = timeseries.value

hover = HoverTool(
        tooltips=[
            ("Value", "$y")
        ]
    )

# create a new plot with a specific size
p = figure(
    title=f"YouTube Trends ({keywords[0].title()}) \n",
    width=700,
    height=350,
    x_axis_label="",
    y_axis_label="",
    tools=[hover]
)



# add line renderer
circle = p.line(x, y)

# datetime custom format
p.xaxis[0].formatter = DatetimeTickFormatter(days=["%b %d, %Y"])

# column scaling
# column(p, sizing_mode='scale_width')

# show the results
show(p)

In [7]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider

x = timeseries.datetime
y = timeseries.value

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(plot_width=700, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")

update_timeframe = CustomJS()

slider.js_on_change('value', update_curve)


show(column(slider, plot))

In [8]:
import numpy as np

from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import CheckboxGroup, CustomJS
from bokeh.palettes import Viridis3
from bokeh.plotting import figure

p = figure()
props = dict(line_width=4, line_alpha=0.7)
x = np.linspace(0, 4 * np.pi, 100)
l0 = p.line(x, np.sin(x), color=Viridis3[0], legend_label="Line 0", **props)
l1 = p.line(x, 4 * np.cos(x), color=Viridis3[1], legend_label="Line 1", **props)
l2 = p.line(x, np.tan(x), color=Viridis3[2], legend_label="Line 2", **props)

checkbox = CheckboxGroup(labels=["Line 0", "Line 1", "Line 2"],
                         active=[0, 1, 2], width=100)

callback = CustomJS(args=dict(l0=l0, l1=l1, l2=l2, checkbox=checkbox), code="""
l0.visible = 0 in checkbox.active;
l1.visible = 1 in checkbox.active;
l2.visible = 2 in checkbox.active;
""")

checkbox.js_on_change('active', callback)

layout = row(checkbox, p)
show(layout)

In [9]:
import datetime
from os.path import dirname, join

import pandas as pd
from scipy.signal import savgol_filter

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure
from bokeh.io import show


timeframes = [
    'today 5-y',
    'today 3-y',
    'today 12-m',
    'today 5-m',
    'today 3-m', 
    'today 1-m',
    'now 1-d',
    'now 7-d',
    'now 1-H',
    'now 4-H'
]

activity_list = timeframes
# activity_list=df['Activity'].unique().tolist().copy()
timeseries['datetime']=pd.to_datetime(timeseries['datetime'])

activity_selected='A'

def modify_doc(doc):
    df_r = timeseries.copy()
    x = df_r.datetime
    y = df_r.value
    source = ColumnDataSource(data=df_r)
    plot = figure(title='Daily Hours', x_axis_type="datetime")
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)
    
    def update_plot(attr, old, new):
        activity =  select.value
        data = [df_r['value'] == activity]
        source.data = ColumnDataSource(data=data).data

    select = Select(title='Select Activity', value='today 5-y', options=activity_list)
    select.on_change('value', update_plot)

    layout=column(select, plot)
    doc.add_root(layout)
    
show(modify_doc)

In [10]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider

x = [x*0.005 for x in range(0, 201)]

source = ColumnDataSource(data=dict(x=x, y=x))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)


show(column(slider, plot))

In [11]:
from bokeh.plotting import figure
from bokeh import events
from bokeh.models import CustomJS, Div, Button
from bokeh.layouts import column, row

import numpy as np
x = np.random.random(size=2000) * 100
y = np.random.random(size=2000) * 100

p = figure(tools="box_select")
p.scatter(x, y, radius=1, fill_alpha=0.6, line_color=None)

div = Div(width=400)
button = Button(label="Button", width=300)
layout = column(button, row(p, div))

# Events with no attributes
button.js_on_event(events.ButtonClick,  CustomJS(args=dict(div=div), code="""
div.text = "Button!";
""")) 

p.js_on_event(events.SelectionGeometry, CustomJS(args=dict(div=div), code="""
div.text = "Selection! <p> <p>" + JSON.stringify(cb_obj.geometry, undefined, 2);
"""))

show(layout)

In [12]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

p = figure(title="simple line example", height=300, width=600, y_range=(-5,5),
           background_fill_color='#efefef')
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

def update(f, w=1, A=1, phi=0):
    if f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()

show(p, notebook_handle=True)

Loading BokehJS ...

In [13]:
interact(update, f=["sin", "cos"], w=(0,50), A=(1,10), phi=(0, 20, 0.1))

<function __main__.update(f, w=1, A=1, phi=0)>

In [14]:
tf = [
    'today 5-y',
    'today 3-y',
    'today 12-m',
    'today 5-m',
    'today 3-m', 
    'today 1-m',
    'now 1-d',
    'now 7-d',
    'now 1-H',
    'now 4-H'
]

In [15]:
from numpy.random import random

from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select, TextInput

def get_data():
    return dict(x=timeseries.datetime, y=timeseries.value)
    pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=tf[4])
    ts = pytrends.interest_over_time()
    ts.reset_index(inplace=True)
    ts.rename(columns={'date':'datetime', keywords[0]: 'value', 'isPartial':'partial'}, inplace=True)
    print(ts)
    return ts

def modify_doc(doc):
    source = ColumnDataSource(data=get_data())

    # p = figure(toolbar_location=None)
    p = figure(
        title=f"YouTube Trends ({keywords[0].title()}) \n",
        width=700,
        height=350,
        x_axis_label="",
        y_axis_label="",
        tools=[]
    )
    r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)
    p.xaxis[0].formatter = DatetimeTickFormatter(days=["%b %d, %Y"])
 
    select = Select(title="Timeframe", value="today 3-m", options=timeframes)
    
    def update_datax(attrname, old, new):
        pytrends.build_payload(keywords, geo=geo, gprop=gprop, timeframe=select.value)
        ts2 = pytrends.interest_over_time()
        ts2.reset_index(inplace=True)
        ts2.rename(columns={'date':'datetime', keywords[0]: 'value', 'isPartial':'partial'}, inplace=True)
        source.data = dict(x=ts2.datetime, y=ts2.value)
        print(ts2)
    select.on_change('value', update_datax)

    # def update_points(attrname, old, new):
    #     N = int(input.value)
    #     source.data = get_data(N)
    # input.on_change('value', update_points)

    layout = column(row(select, width=300), row(p))

    doc.add_root(layout)

show(modify_doc)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "x", key "y" value "y" [renderer: GlyphRenderer(id='1885', ...)]
